In [ ]:
import io
import matplotlib as mpl
mpl.use("Agg")

import logging
logging.getLogger('tensorflow').disabled = True

import matplotlib.pyplot as plt
import numpy as np
import re
import math
import tensorflow as tf
import random

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

from sklearn.feature_extraction.text import CountVectorizer


tf.random.set_seed(123)
np.random.seed(123)

special_chars_remover = re.compile("[^\w'|_]")

# 특수 문자 제거
def remove_special_characters(sentence):
    return special_chars_remover.sub(' ', sentence)

# data/ratings.txt 에서 데이터를 읽어, 인공신경망 학습을 위한 두 개의 리스트를 반환
def read_data():
    sentences = []
    labels = []
    with open("data/ratings.txt") as fp:
        next(fp)
        for line in fp:
            sentence, label = line.split('\t')[1:]
            sentences.append(remove_special_characters(sentence))
            labels.append(int(label))
    print(labels)

    return sentences,labels


def count_vect(sentences, testing_sentence):
    
    # 테스트 문장 또한 토큰 빈도수 안에 포함되어야하기 때문에 sentences 리스트에 추가
    sentences.append(testing_sentence)
    
    # sentences를 카운트 벡터로 변환
    Vectorizer = CountVectorizer(min_df=1) #게시물마다 등장하는 단어의 빈도수를 파악해 하나의 카운트 벡터로 만든다. min_df값보다 작은 빈도수를 가진 단어는 제외된다.
    vector = Vectorizer.fit_transform(sentences) #실제 변환
    vector = vector.toarray()
    
    return vector


def ANN(vector,labels):
    
    # 카운트 벡터로 변환된 테스트 문장 벡터를 저장
    test = vector[-1]
    # 모델 학습 데이터에서 테스트 데이터를 제거
    vector = vector[:-1]
    # 모델 입력을 위한 형태로 변환
    test = [[test]]
    
    # 입력 데이터의 차원은 카운트 벡터 안의 토큰 수
    num_voca = len(vector[0])
    
    # 인공 신경망 생성
    ANN_model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(20, input_dim = num_voca, activation='relu'),
        tf.keras.layers.Dense(20, activation='relu'),
        tf.keras.layers.Dense(2,activation='softmax')
    ])

    # loss와 optimizer를 설정하세요.
    ANN_model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
    ANN_model.fit(vector, labels , epochs=500, verbose=0)
    predict = ANN_model.predict(test)
    
    return predict

def main():
    
    train_sentences,labels = read_data()
    
    testing_sentence = "어설픈 연기들로 몰입이 전혀 안되네요"
        
    bow_vect = count_vect(train_sentences, testing_sentence)
    probs = ANN(bow_vect,labels)
    
    # 시각화
    plot_title = testing_sentence
    if len(plot_title) > 50: plot_title = plot_title[:50] + "..."
    visualize_boxplot(plot_title,
                  [probs[0][0],probs[0][1]],
                  ['Negative', 'Positive'])

def visualize_boxplot(title, values, labels):
    width = .35

    print(title)
    
    fig, ax = plt.subplots()
    ind = np.arange(len(values))
    rects = ax.bar(ind, values, width)
    ax.bar(ind, values, width=width)
    ax.set_xticks(ind + width/2)
    ax.set_xticklabels(labels)

    def autolabel(rects):
        # attach some text labels
        for rect in rects:
            height = rect.get_height()
            ax.text(rect.get_x()+rect.get_width()/2., height + 0.01, '%.2lf%%' % (height * 100), ha='center', va='bottom')

    autolabel(rects)

    plt.savefig("image.svg", format="svg")


if __name__ == "__main__":
    main()
